In [1]:
import pandas as pd

# Establishing a timeline of sofware versions among user base

Suppose we have a set of software users, uniquely identified by a `user_id`. At any given time, each user has version `v1`, `v2` or `v3` of the software. In a database, we record timestamps of individual software updates.

We study the time period ranging the first 10 days of January 2018, assuming that:
- the user base is stable (no users join or quit)
- all users start on January 1st with version `v1`

We would like to plot, at any given day, the repartition of versions among the user base. For clarity, in the example below we consider only 3 users `a`, `b` and `c`.

In [52]:
dates = [
    pd.to_datetime('2018-01-01 00:00:00'),  # user a:
    pd.to_datetime('2018-01-03 00:25:00'),  #   v2 upgrade
    pd.to_datetime('2018-01-05 00:10:00'),  #   v3 upgrade
    pd.to_datetime('2018-01-01 00:00:00'),  # user b:
    pd.to_datetime('2018-01-06 00:15:00'),  #   v2 upgrade
    pd.to_datetime('2018-01-09 23:00:00'),  #   v3 upgrade
    pd.to_datetime('2018-01-01 00:00:00'),  # user c:
    pd.to_datetime('2018-01-04 00:16:00'),  #   v2 upgrade
    pd.to_datetime('2018-01-08 11:00:00'),  #   v3 upgrade
]
data = {
    'upgrade_date': dates,
    'version': ['v1', 'v2', 'v3'] * 3,
    'user_id': ['a'] * 3 + ['b'] * 3 + ['c'] * 3,
}
df = pd.DataFrame(data).set_index('upgrade_date').sort_index();df

,user_id,version
upgrade_date,,
2018-01-01 00:00:00,a,v1
2018-01-01 00:00:00,b,v1
2018-01-01 00:00:00,c,v1
2018-01-03 00:25:00,a,v2
2018-01-04 00:16:00,c,v2
2018-01-05 00:10:00,a,v3
2018-01-06 00:15:00,b,v2
2018-01-08 11:00:00,c,v3
2018-01-09 23:00:00,b,v3


So for example, according to the data sample above, user `a` upgraded to version `v2` on January 3rd about half an hour after midnight. 

## Adding a common last date for each user

In [53]:
LAST_DATE = pd.to_datetime('2018-01-10 00:00:00')
def add_last_date(group):
    group.loc[LAST_DATE] = group.iloc[-1]
    return group
df = df.groupby('user_id', group_keys=False).apply(add_last_date);df

,user_id,version
upgrade_date,,
2018-01-01 00:00:00,a,v1
2018-01-03 00:25:00,a,v2
2018-01-05 00:10:00,a,v3
2018-01-10 00:00:00,a,v3
2018-01-01 00:00:00,b,v1
2018-01-06 00:15:00,b,v2
2018-01-09 23:00:00,b,v3
2018-01-10 00:00:00,b,v3
2018-01-01 00:00:00,c,v1


## Rounding the time index to days

We are interested in daily statistics, so the next step is to round the timestamps at day precision.

In [54]:
df.index = df.index.round('D');df

,user_id,version
upgrade_date,,
2018-01-01,a,v1
2018-01-03,a,v2
2018-01-05,a,v3
2018-01-10,a,v3
2018-01-01,b,v1
2018-01-06,b,v2
2018-01-10,b,v3
2018-01-10,b,v3
2018-01-01,c,v1


## Filling the time ranges

Now we create a daily timeline for each user ranging from January 1st to January 10th, with the corresponding versions.
Below, ** ffill ** is used to forward fill the values that have been resampled, i.e fill the gaps based on preceding values.

In [70]:
dfa = df.groupby('user_id', group_keys=False).resample('D').ffill();dfa

,user_id,version
upgrade_date,,
2018-01-01,a,v1
2018-01-02,a,v1
2018-01-03,a,v2
2018-01-04,a,v2
2018-01-05,a,v3
2018-01-06,a,v3
2018-01-07,a,v3
2018-01-08,a,v3
2018-01-09,a,v3


Note: We could also have used the default ** groupby ** behavior, which would have added "user_id" as an index, hence giving us a multi-indexed DataFrame.

In [63]:
dfb = df.groupby('user_id').resample('D').ffill();dfb

user_id version
user_id upgrade_date                
a       2018-01-01         a      v1
        2018-01-02         a      v1
        2018-01-03         a      v2
        2018-01-04         a      v2
        2018-01-05         a      v3
        2018-01-06         a      v3
        2018-01-07         a      v3
        2018-01-08         a      v3
        2018-01-09         a      v3
        2018-01-10         a      v3
b       2018-01-01         b      v1
        2018-01-02         b      v1
        2018-01-03         b      v1
        2018-01-04         b      v1
        2018-01-05         b      v1
        2018-01-06         b      v2
        2018-01-07         b      v2
        2018-01-08         b      v2
        2018-01-09         b      v2
        2018-01-10         b      v3
c       2018-01-01         c      v1
        2018-01-02         c      v1
        2018-01-03         c      v1
        2018-01-04         c      v2
        2018-01-05         c      v2
        2018-01-06         c      v2
        2018-01-07         c      v2
        2018-01-08         c      v3
        2018-01-09         c      v3
        2018-01-10         c      v3

## Counting versions for each given date

Next, we count the number of versions at each given date 

In [72]:
versions = dfa['version'].groupby(level=0).apply(lambda x: x.value_counts())
# Equivalent
versions = dfb['version'].groupby(level=1).apply(lambda x: x.value_counts()); versions

upgrade_date    
2018-01-01    v1    3
2018-01-02    v1    3
2018-01-03    v1    2
              v2    1
2018-01-04    v2    2
              v1    1
2018-01-05    v3    1
              v2    1
              v1    1
2018-01-06    v2    2
              v3    1
2018-01-07    v2    2
              v3    1
2018-01-08    v3    2
              v2    1
2018-01-09    v3    2
              v2    1
2018-01-10    v3    3
Name: version, dtype: int64

It would be more convenient to have version numbers in columns, rather than as an extra level in the series index as is the case above. This is done by 'unstacking':

In [73]:
versions = versions.unstack(level=1).fillna(0).astype(int); versions

,v1,v2,v3
upgrade_date,,,
2018-01-01,3,0,0
2018-01-02,3,0,0
2018-01-03,2,1,0
2018-01-04,1,2,0
2018-01-05,1,1,1
2018-01-06,0,2,1
2018-01-07,0,2,1
2018-01-08,0,1,2
2018-01-09,0,1,2


As a sanity check, we can make sure that the total number of users for each date is 3.

In [76]:
assert (versions.sum(axis=1) == 3).all()

# Plotting the results with Bokeh

Now the data is ready for plotting.

In [78]:
import bokeh
from bokeh.plotting import output_notebook, figure, show, ColumnDataSource
from bokeh.core.properties import value
output_notebook()

Loading BokehJS ...

In [79]:
source = ColumnDataSource(versions)
p = figure(
    title='Version counts',
    width=800,
    height=600,
    x_axis_type='datetime'
)

stacks = 'v3 v2 v1'.split()
colors = 'red yellow green'.split()
p.vbar_stack(
    stacks,
    x='upgrade_date',
    color=colors,
    width=pd.to_timedelta('6h'),
    source=source,
    legend=[value(v) for v in stacks]
)

show(p)